In [ ]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import torch 
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import random

import pandas as pd
import re

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(2022)
if device == 'cuda':
    torch.cuda.manual_seed_all(2022)

In [ ]:
#파일 불러오기
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv', encoding = 'utf-8') #한글의 경우 encoding으로 utf-8, ms949,cp949
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv', encoding = 'utf-8')

#### 전처리

In [ ]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)
test['text']=test['text'].apply(alpha_num)

In [ ]:
import nltk
from nltk.corpus import stopwords 

nltk.download('stopwords')
stopwords = stopwords.words('english')

print(stopwords[:10])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [ ]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

In [ ]:
def remove_stopwords(text):
    final_text = []
    words = tokenizer.tokenize(text)
    for word in words:
        if word.strip().lower() not in stopwords:
          final_text.append(word.strip())
    return  " ".join(final_text)


train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords) #전처리한 test에 불용어 처리 함수 적용
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer(max_features = 1000) # 1000개로 지정 -> 더 크게한다면 더 좋을 듯
v.fit(train['text'])

TfidfVectorizer(max_features=1000)

In [ ]:
word_set = []
max_len = 0

for d in train['text']:
  word_set = word_set + d.split(' ') 
  if len(d.split()) > max_len:
    max_len = len(d.split())
  
word_set = set(word_set) 

In [ ]:
word_to_idx = {word: i+1 for i, word in enumerate(word_set)} # 사전 만들기
print(len(word_set))
print(max_len)

47120
212


In [ ]:
def word_to_key(text):
  final_text = []
  for word in text.split():
      final_text.append(word_to_idx[word]) # 만든 사전에서 인덱스 번호 지정
  if len(final_text) < max_len:
    final_text = final_text + [0] * (max_len - len(final_text)) #패딩
  return final_text


train['word_to_key'] = train['text'].apply(word_to_key)

In [ ]:
train

,index,text,author,word_to_key
0,0,almost choking much much wanted say strange ex...,3,"[6709, 29296, 15510, 15510, 15220, 10621, 3531..."
1,1,sister asked suppose,2,"[25889, 12146, 28879, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,engaged one day walked perusing janes last let...,1,"[32881, 1017, 14396, 41742, 12643, 16750, 3157..."
3,3,captain porch keeping carefully way treacherou...,4,"[15058, 24581, 35234, 38733, 42068, 26575, 180..."
4,4,mercy gentlemen odin flung hands dont write an...,3,"[32176, 3058, 6099, 41025, 5578, 45738, 16829,..."
...,...,...,...,...
54874,54874,mr smith odin whispered hardly dared hope woul...,2,"[16189, 38458, 6099, 39754, 8565, 8927, 42484,..."
54875,54875,told plan captain us settled details accomplis...,4,"[18875, 5427, 15058, 42916, 14243, 46714, 3822..."
54876,54876,sincere wellwisher friend sister lucy odin,1,"[23287, 2164, 5539, 25889, 7710, 6099, 0, 0, 0..."
54877,54877,wanted lend money,3,"[15220, 727, 21454, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


#### 모델링

In [ ]:
X_train = train.iloc[:45000, 3]
X_test = train.iloc[45000:, 3].reset_index(drop=True)

y_train = train.iloc[:45000, 2]
y_test = train.iloc[45000:, 2].reset_index(drop=True)

In [ ]:
# 하이퍼파라미터
batch_size = 64
lr = 0.001
epochs = 20 

In [ ]:
class CustomDataset(Dataset):
  def __init__(self):
    
    self.x_data = X_train
    self.y_data = [[y] for y in y_train]

  def __len__(self):

    return len(self.x_data)

  def __getitem__(self, idx):

    x = torch.LongTensor(self.x_data[idx]).to(device)
    y = torch.LongTensor(self.y_data[idx]).to(device)

    return x,y

In [ ]:
class CustomDataset_test(Dataset):
  def __init__(self):
    
    self.x_data = X_test
    self.y_data = [[y] for y in y_test]

  def __len__(self):

    return len(self.x_data)

  def __getitem__(self, idx):

    x = torch.LongTensor(self.x_data[idx])
    y = torch.LongTensor(self.y_data[idx])

    return x,y

In [ ]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=batch_size) 

In [ ]:
class RNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
      super(RNN, self).__init__()
      self.n_layers = n_layers
      self.hidden_dim = hidden_dim

      self.embed = nn.Embedding(n_vocab, embed_dim)
      self.dropout = nn.Dropout(dropout_p)
      self.gru = nn.GRU(embed_dim, self.hidden_dim,
                        num_layers=self.n_layers,
                        batch_first=True)
      #self.rnn = nn.RNN(embed_dim, self.hidden_dim,batch_first = True)
      self.out = nn.Sequential(
          nn.Linear(self.hidden_dim, n_classes),
          nn.Softmax()
      )
    def forward(self, x):
      x = self.embed(x)
      h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
      x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
      #x, _ = self.rnn(x,h_0)
      h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
      self.dropout(h_t)
      logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
      return logit

    def _init_state(self, batch_size=1):
      weight = next(self.parameters()).data
      return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [ ]:
n_vocab = 47120+1
embedd_size = 100
hidden_size = 500
output_size = 5

In [ ]:
net = RNN(5, 256, n_vocab, embedd_size, output_size, 0.5).to(device) #GRU층 5개 쌓기

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr)

In [ ]:
losses = []
for epoch in range(epochs):
  
  for x, y in dataloader:
    optimizer.zero_grad()
    y = y.to(device)

    # forward 연산
    hypothesis = net(x)

    # 비용 함수
    y = y.squeeze()
    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()
    losses.append(cost.item()) # 값만 가져오기 위해서 .item()

  # 10의 배수에 해당되는 에포크마다 비용을 출력
  if epoch % 2 == 0:
      print(epoch, cost.item())

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0 1.3959771394729614
2 1.3953770399093628
4 1.292704701423645
6 1.272382378578186
8 1.1560006141662598
10 1.2679107189178467
12 1.2711976766586304
14 1.030347228050232
16 1.1548601388931274
18 1.0298491716384888


RNN층 하나를 썼을 때보다는 loss가 확연히 줄어듦

In [ ]:
dataset = CustomDataset_test()
test_loader = DataLoader(dataset, batch_size=batch_size)

In [ ]:
correct = 0

with torch.no_grad():
  net = net.to('cpu')
  net.eval()
  for data, target in test_loader:
    data, target = data, target
    output = net(data)
    
    pred = output.max(1, keepdim=True)[1]
    # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
    correct += pred.eq(target.view_as(pred)).sum().item()

test_accuracy = correct / len(test_loader.dataset)
print('Accuracy:', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Accuracy: 0.6150420083004353


분반장님이 준비하신거에 비해 매우 초라한... 이번 과제다. 사실 일정들이 여러 개 겹쳐서 시간이 없긴 했지만 핑계다.. 이번 과제에서는 코드 이해와 몇몇 파라미터를 바꿔가면서 시도를 해보았다. 전처리로는 정규 표현식을 통한 가공과 불용어 처리를 해주었고 모델로는 RNN 아키텍쳐를 이용했다. 층이 여러개 쌓으니 loss가 줄어들었으나 10개 층일 떄보다 5개의 층일 때 학습이 더 잘 되었다.(더 여려가지를 시도해보지 않아서 정확하지는 않다.)  
실행 시간의 문제 때문에 단어 벡터의 개수를 1000보다 더 늘리고 적절한 층의 개수와 학습률을 찾는다면 더 개선될 것 같다.